In [134]:
# Importing the Libraries

from tensorflow.keras.models import load_model
import numpy as np
import pickle
import heapq
import random
from random import randrange
from tensorflow.keras.preprocessing.sequence import pad_sequences

def model_selection():
    
    seq_length =  {'Autos & Vehicles':19
                  ,'Comedy':20
                  ,'Education':16
                  ,'Entertainment':23
                  ,'Film & Animation':17
                  ,'Gaming':18
                  ,'Howto & Style':22
                  ,'Music':21.  
                  ,'News & Politics':18
                  ,'Nonprofits & Activism':16
                  ,'People & Blogs':20
                  ,'Pets & Animals':18
                  ,'Science & Technology':16
                  ,'Sports':21
                  ,'Travel & Events':17}
    
    selected = False
    while selected != True:
        print("Categories available are Autos & Vehicles, Comedy, Education, Entertainment, Film & Animation, Gaming, Howto & Style, Music, News & Politics, Nonprofits & Activism, People & Blogs, Pets & Animals, Science & Technology,Sports, and Travel & Events")
        category = input("Please Select a Category From the List")
        
        if category in ('Autos & Vehicles','Comedy','Education','Entertainment','Film & Animation','Gaming','Howto & Style',
                        'Music','News & Politics','Nonprofits & Activism','People & Blogs','Pets & Animals','Science & Technology',
                        'Sports','Travel & Events'):
            selected = True
            model = load_model(category+'keras_next_word_model.h5')
            tokenizer = pickle.load(open(category+'tokenizer.pkl', 'rb'))
            slen = seq_length[category]
            return model, tokenizer, slen
        else:
            print("Not a possible selection")

def sample(preds, top_n):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.argpartition(preds[0], -5)[-5:]

def predict_completions(text, n, unique_words, used_words):
    preds = model.predict(text)
    next_indices = sample(preds, n + 25)
    return [unique_words[idx] for idx in next_indices if unique_words[idx] not in used_words][:n]

def Predict_Next_Words(model, tokenizer, text, used_words, slen):

    #flip dictionary keys to values
    lookupvalue = {v: k for k, v in tokenizer.word_index.items()}

    sequence = np.array(tokenizer.texts_to_sequences([text])[-1])

    pad_encoded = pad_sequences([sequence], maxlen=slen, truncating='pre')

    preds = predict_completions(pad_encoded, 5, lookupvalue, used_words)

    return preds

def generate_random_title(word_length, model, tokenizer, slen):
    first_word = input("Input a word to work off for the title: ")
#     first_word = random.choice(list(tokenizer.word_index.keys()))
    titles = []
    for i in range(5):
        used_words = [i for i in first_word.split(' ')]
        for i in range(word_length):
            text = Predict_Next_Words(model, tokenizer, used_words[-1], used_words, slen)
            used_words.append(text[randrange(len(text))])
        titles.append(' '.join(used_words))
    return titles

In [136]:
model, tokenizer, slen = model_selection()

Categories available are Autos & Vehicles, Comedy, Education, Entertainment, Film & Animation, Gaming, Howto & Style, Music, News & Politics, Nonprofits & Activism, People & Blogs, Pets & Animals, Science & Technology,Sports, and Travel & Events


Please Select a Category From the List Comedy


In [137]:
generate_random_title(7, model, tokenizer, slen)

Input a word to work off for the title:  Pokemon players in the wild


1/1 [==============================] - 0s 34ms/step


/Users/garrisonwinter/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: divide by zero encountered in log


1/1 [==============================] - 0s 64ms/step


['Pokemon players in the wild singh gives youre completely worst construit polystyrene',
 'Pokemon players in the wild removal jimin ever v episode speedrun us',
 'Pokemon players in the wild singh blows are transformation corn service g',
 'Pokemon players in the wild snl misadventures always tiktokers gucci reigns ropes',
 'Pokemon players in the wild snl misadventures always sisters random how asian']